In [1]:
import pandas as pd
import numpy as np
import ast
import json

In [2]:
#Cargamos la base de datos 'movies_dataset'
df_mo = pd.read_csv("C:/Users/guard/OneDrive/Desktop/Henry Data Science/Proyecto MLOps/Datos/movies_dataset.csv", sep=",")

df_mo.head(2)

C:\Users\guard\AppData\Local\Temp\ipykernel_2644\2305798667.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mo = pd.read_csv("C:/Users/guard/OneDrive/Desktop/Henry Data Science/Proyecto MLOps/Datos/movies_dataset.csv", sep=",")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
df_mo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

 Se observa que el tipo de la columna 'budget' es object, vamos a revisar uno a uno los valores de la columna para convertirlos en  numéricos

In [4]:
# Función para verificar si un valor es numérico
def verificar_numerico(valor):
    try:
        # Intentamos convertir el valor a float
        float(valor)
        return True
    except (ValueError, TypeError):
        return False

# Iterar sobre la columna 'budget' y reemplazar valores no numéricos con 0
for index, value in df_mo['budget'].items():
    if not verificar_numerico(value):
        df_mo.at[index, 'budget'] = 0  # Reemplazar con 0 si no es numérico

# Convertir la columna a float64
df_mo['budget'] = df_mo['budget'].astype(float)

# Revisamos nuevamente el tipo de los datos
df_mo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  float64
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
# Reemplazar valores nulos en la columna 'revenue' con 0
df_mo['revenue'] = df_mo['revenue'].fillna(0)

In [6]:
# Eliminar filas donde 'release_date' es nulo
df_mo = df_mo.dropna(subset=['release_date'])

In [7]:
# Convertir la columna 'release_date' al formato de fecha AAAA-mm-dd
df_mo['release_date'] = pd.to_datetime(df_mo['release_date'], format='%Y-%m-%d', errors='coerce')

In [8]:
# Crear la nueva columna 'release_year' extrayendo el año de 'release_date'
df_mo['release_year'] = df_mo['release_date'].dt.year

# Verificar el resultado
print(df_mo[['release_date', 'release_year']])

      release_date  release_year
0       1995-10-30        1995.0
1       1995-12-15        1995.0
2       1995-12-22        1995.0
3       1995-12-22        1995.0
4       1995-02-10        1995.0
...            ...           ...
45460   1991-05-13        1991.0
45462   2011-11-17        2011.0
45463   2003-08-01        2003.0
45464   1917-10-21        1917.0
45465   2017-06-09        2017.0

[45379 rows x 2 columns]


In [9]:
# Crear la columna 'return' dividiendo 'revenue' entre 'budget'
df_mo['return'] = np.where(df_mo['budget'].fillna(0) > 0, 
                            df_mo['revenue'] / df_mo['budget'], 
                            0)

# Verificar el resultado
print(df_mo[['revenue', 'budget', 'return']])

           revenue      budget     return
0      373554033.0  30000000.0  12.451801
1      262797249.0  65000000.0   4.043035
2              0.0         0.0   0.000000
3       81452156.0  16000000.0   5.090760
4       76578911.0         0.0   0.000000
...            ...         ...        ...
45460          0.0         0.0   0.000000
45462          0.0         0.0   0.000000
45463          0.0         0.0   0.000000
45464          0.0         0.0   0.000000
45465          0.0         0.0   0.000000

[45379 rows x 3 columns]


In [10]:
# Eliminar las columnas especificadas
df_mo = df_mo.drop(columns=['belongs_to_collection','production_companies', 'production_countries', 'spoken_languages', 'video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'status', 'tagline', 'overview'])

In [11]:
# Verificar el resultado
df_mo.head(2)

,budget,genres,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995.0,12.451801
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,1995.0,4.043035


Vemos que la columna 'genres' es una string, pero también tiene formato de diccionario, y más adelante la vamos a necesitar para el sistema de recomendación, entonces vamos a desagregarlos convirtiendo cada género en una columna binaria.

In [13]:
# Función para intentar evaluar una string como lista de diccionarios
def safe_eval(entry):
    if isinstance(entry, list):
        return entry
    try:
        return ast.literal_eval(entry)
    except (ValueError, SyntaxError):
        return json.loads(entry.replace("'", '"'))

# Aplicar safe_eval a las columnas 'genres'
df_mo['genres'] = df_mo['genres'].apply(safe_eval)

# Extraer los nombres de genres
df_mo['genres_1'] = df_mo['genres'].apply(lambda x: ', '.join(d['name'] for d in x) if isinstance(x, list) else '')



In [14]:
# Función para intentar evaluar una string como lista de diccionarios
def safe_eval(entry):
    if isinstance(entry, list):
        return entry
    try:
        return ast.literal_eval(entry)
    except (ValueError, SyntaxError):
        return json.loads(entry.replace("'", '"'))

# Aplicar safe_eval a la columna 'genres'
df_mo['genres'] = df_mo['genres'].apply(safe_eval)

# Crear una nueva columna 'genre_names' para almacenar los nombres de los géneros
df_mo['genre_names'] = df_mo['genres'].apply(lambda x: [d['name'] for d in x] if isinstance(x, list) else [])

# Explota la lista de géneros en filas individuales
df_expanded = df_mo.explode('genre_names')

# Utiliza get_dummies para crear columnas separadas para cada género
df_with_dummies = pd.get_dummies(df_expanded, columns=['genre_names'], prefix='', prefix_sep='')

# Seleccionar las columnas que no deben ser agrupadas (mantener las de la primera fila)
columns_to_keep = ['budget', 'genres', 'id', 'original_language', 'popularity', 'release_date', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'release_year', 'return']

# Mantener los valores de la primera fila para las columnas seleccionadas
df_first_rows = df_with_dummies.groupby(df_with_dummies.index).first()[columns_to_keep]

# Seleccionar solo las columnas binarias generadas por get_dummies para sumar
columns_to_sum = df_with_dummies.columns.difference(columns_to_keep)

# Sumar los valores de las columnas dummies
df_summed = df_with_dummies[columns_to_sum].groupby(df_with_dummies.index).sum()

# Combinar ambos DataFrames
df_final = df_first_rows.join(df_summed)


In [15]:
# Muestra el DataFrame resultante luego de extraer los géneros
df_final.head(2)

,budget,genres,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,...,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western,genres_1
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,...,0,0,0,0,0,0,0,0,0,"Animation, Comedy, FamilyAnimation, Comedy, Fa..."
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,...,0,0,0,0,0,0,0,0,0,"Adventure, Fantasy, FamilyAdventure, Fantasy, ..."


In [16]:
# Ya extraídos los géneros eliminamos la columna 'genres'"ETL genres movies_dataset.ipynb"

df_final = df_final.drop(columns=['genres'])

In [17]:
df_final.head(2)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western,genres_1
0,30000000.0,862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,0,0,0,0,0,0,0,0,0,"Animation, Comedy, FamilyAnimation, Comedy, Fa..."
1,65000000.0,8844,en,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,...,0,0,0,0,0,0,0,0,0,"Adventure, Fantasy, FamilyAdventure, Fantasy, ..."


In [18]:
# Exportar el DataFrame a un archivo CSV
df_final.to_csv('movies_dataset_procesado.csv', index=False)  # index=True incluye el índice en el archivo CSV